# Extract Embeddings
This notebook is for building and testing a component for reading content from context, extracting their embeddings and putting the embeddings into the context.

Prequisites:
- Install and start the node-engine-service based on instructions in [../../README.md](../../README.md)

In [20]:
components_path = "../components/"

# register a magic command to save a cell to a file
from IPython.core.magic import register_cell_magic
import os

@register_cell_magic
def run_and_save(line, cell):
    # create directory if it does not exist
    if not os.path.exists(components_path):
        os.makedirs(components_path)

    'Run and save python code block to a file'
    path = components_path + line
    with open(path, 'wt') as fd:
        fd.write(cell)
    code = compile(cell, path, 'exec')
    exec(code, globals())

In [21]:
%%run_and_save extract_embeddings.py
# Copyright (c) Microsoft. All rights reserved.

from node_engine.libs.azure_openai_embeddings import AzureOpenAIEmbeddings
from node_engine.libs.node_engine_component import NodeEngineComponent
from node_engine.models.flow_step import FlowStep


class ExtractEmbeddings(NodeEngineComponent):
    description = "Extract embeddings from text. Pass the text in the context and the embeddings will be returned in the context."

    default_config = {"service": "AZUREOPENAI_EMBEDDINGS"}

    reads_from = {
        "context": "as defined in config",
        "config": {
            "source": {
                "type": "string",
                "description": "The key in the context containing the text to be embedded.",
                "required": True,
            },
            "target": {
                "type": "string",
                "description": "The key in the context to store the embeddings.",
                "required": True,
            },
        },
    }

    writes_to = {
        "context": "as defined in config",
    }

    sample_input = {
        "key": "sample",
        "session_id": "123456",
        "context": {
            "memories": [
                "Offering help: Assistant offered help and encouraged the user to ask questions if needed."
            ],
        },
        "flow": [
            {
                "key": "start",
                "name": "ExtractEmbeddings",
                "config": {
                    "source": "memories",
                    "target": "memories_embeddings",
                },
            },
        ],
    }

    sample_output = {
        "key": "sample",
        "session_id": "123456",
        "context": {
            "memories": [
                "Offering help: Assistant offered help and encouraged the user to ask questions if needed."
            ],
            "memories_embeddings": [
                {
                    "item": "Offering help: Assistant offered help and encouraged the user to ask questions if needed.",
                    "embedding": [
                        -0.0003509521484375,
                        0.00018310546875,
                        0.0001220703125,
                    ],
                }
            ],
        },
    }

    azure_openai_embeddings = AzureOpenAIEmbeddings()

    async def execute(self) -> FlowStep:
        # extract embeddings

        self.log("Extract embeddings started")

        # get source from context
        source_key = self.config.get("source")
        if not source_key:
            return self.exit_flow_with_error("Source not found in config")
        source_data = self.context.get(source_key)
        if not source_data:
            return self.exit_flow_with_error(
                "Source '{}' not found in context".format(source_key)
            )

        # get target from context
        target_key = self.config.get("target")
        if not target_key:
            return self.exit_flow_with_error("Target not found in config")

        service = self.config.get("service") or self.default_config["service"]

        # get embeddings
        embeddings = await self.azure_openai_embeddings.create(source_data, service)

        results = []
        for i in range(len(source_data)):
            results.append({"item": source_data[i], "embedding": embeddings[i]})

        # add embeddings to context
        self.context.set(target_key, results)

        self.log("Extract embeddings completed")

        return self.continue_flow()


In [22]:
# Test it
from node_engine.client import RemoteExecutor

result = await ExtractEmbeddings.test(executor=RemoteExecutor())
display(result.model_dump())

[16:15:09] INFO     node_engine:ExtractEmbeddings:start | s:123456 | f:sample | Extract embeddings        ]8;id=50444;file:///Users/mark/src/github.com/microsoft/nodeengine/node_engine/libs/log.py\log.py]8;;\:]8;id=338644;file:///Users/mark/src/github.com/microsoft/nodeengine/node_engine/libs/log.py#53\53]8;;\
                    started                                                                                        

           INFO     node_engine:ExtractEmbeddings:start | s:123456 | f:sample | Extract embeddings        ]8;id=606506;file:///Users/mark/src/github.com/microsoft/nodeengine/node_engine/libs/log.py\log.py]8;;\:]8;id=409666;file:///Users/mark/src/github.com/microsoft/nodeengine/node_engine/libs/log.py#53\53]8;;\
                    completed                                                                                      

{'next': None,
 'flow_definition': {'key': 'sample',
  'session_id': '123456',
  'flow': [{'key': 'start',
    'name': 'ExtractEmbeddings',
    'config': {'source': 'memories', 'target': 'memories_embeddings'}}],
  'registry': None,
  'context': {'memories': ['Offering help: Assistant offered help and encouraged the user to ask questions if needed.'],
   'memories_embeddings': [{'item': 'Offering help: Assistant offered help and encouraged the user to ask questions if needed.',
     'embedding': [-0.011592058,
      -0.01912604,
      0.031334516,
      0.017670613,
      -0.0121143,
      -0.0052352613,
      -0.0018620922,
      0.046744935,
      0.001261728,
      0.023098504,
      -0.006155606,
      -0.006352517,
      0.018115802,
      -0.012208476,
      -0.011574936,
      0.010607504,
      -0.007165844,
      0.01701139,
      -0.016335042,
      0.029536635,
      0.0020151262,
      0.017687734,
      -0.0029215585,
      0.011925951,
      0.0122940885,
      -0.0040409